In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../scripts/')
sys.path.insert(0, '../../../../dataset/')
import embedding_reader
import custom_keras_metrics
import tensorflow as tf
import keras.backend as K
import os
import time
import gc
import glob
import wget
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing import text, sequence
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import metrics
from tqdm import tqdm

2023-10-23 00:01:22.313045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-23 00:01:22.313084: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#wget.download("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip", out="../../../../embeddings/")

In [3]:
#with zipfile.ZipFile("../../../../embeddings/glove_s300.zip", 'r') as zip_ref:
    #zip_ref.extractall("../../../../embeddings/")

In [2]:
MAX_LENGTH = 300
dataframe = pd.read_csv('../../../../dataset/HateBR.csv')
y = dataframe['offensive_language']

In [3]:
x = dataframe['instagram_comments']

In [4]:
token = Tokenizer()
token.fit_on_texts(x)
sequences = token.texts_to_sequences(x)
x = pad_sequences(sequences,maxlen=MAX_LENGTH)
vocabulary_size = len(token.word_index)+1

In [5]:
embedding_vector = embedding_reader.read_glove_file()
embedding_matrix = np.zeros((vocabulary_size,MAX_LENGTH))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 11530/11530 [00:00<00:00, 362160.10it/s]


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [7]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size,MAX_LENGTH,weights=[embedding_matrix],input_length=x.shape[1],trainable=False))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy', custom_keras_metrics.f1_score])

2023-10-23 00:04:04.057457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-23 00:04:04.057508: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-23 00:04:04.057538: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-fbf4a0): /proc/driver/nvidia/version does not exist
2023-10-23 00:04:04.057863: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model = lstm_model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/15


2023-10-23 00:04:07.939556: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-23 00:04:07.958483: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-23 00:04:08.036189: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.
2023-10-23 00:04:08.135931: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


 1/44 [..............................] - ETA: 1:32 - loss: 0.6748 - accuracy: 0.5703 - binary_accuracy: 0.5703 - f1_score: 0.5926

2023-10-23 00:04:08.638313: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 46080000 exceeds 10% of free system memory.


44/44 [==============================] - 25s 541ms/step - loss: 0.5842 - accuracy: 0.7023 - binary_accuracy: 0.7023 - f1_score: 0.7116 - val_loss: 0.4650 - val_accuracy: 0.7993 - val_binary_accuracy: 0.7993 - val_f1_score: 0.7942
Epoch 2/15
44/44 [==============================] - 24s 541ms/step - loss: 0.4356 - accuracy: 0.8052 - binary_accuracy: 0.8052 - f1_score: 0.8035 - val_loss: 0.3833 - val_accuracy: 0.8336 - val_binary_accuracy: 0.8336 - val_f1_score: 0.8216
Epoch 3/15
44/44 [==============================] - 24s 538ms/step - loss: 0.3855 - accuracy: 0.8291 - binary_accuracy: 0.8291 - f1_score: 0.8272 - val_loss: 0.3594 - val_accuracy: 0.8443 - val_binary_accuracy: 0.8443 - val_f1_score: 0.8368
Epoch 4/15
44/44 [==============================] - 23s 534ms/step - loss: 0.3617 - accuracy: 0.8370 - binary_accuracy: 0.8370 - f1_score: 0.8314 - val_loss: 0.3684 - val_accuracy: 0.8300 - val_binary_accuracy: 0.8300 - val_f1_score: 0.8308
Epoch 5/15
44/44 [=============================

In [9]:
results = lstm_model.evaluate(x_test, y_test, batch_size=128)

for name, value in zip(lstm_model.metrics_names, results):
  print(name, ': ', value)
print()

 1/11 [=>............................] - ETA: 1s - loss: 0.2602 - accuracy: 0.8906 - binary_accuracy: 0.8906 - f1_score: 0.8906

11/11 [==============================] - 2s 168ms/step - loss: 0.3485 - accuracy: 0.8621 - binary_accuracy: 0.8621 - f1_score: 0.8555
loss :  0.3484896123409271
accuracy :  0.8621428608894348
binary_accuracy :  0.8621428608894348
f1_score :  0.8554824590682983



: 

In [12]:
#import joblib

#model_filename = '../../../models/generated-models-base-paper/offensive_lstm_embeddings_model.pkl'
#joblib.dump(model, model_filename)

: 